### Alternative constructors: The simple factory

In [1]:
class Money:
    def __init__(self,dollars,cents):
        self.dollars = dollars
        self.cents = cents

Let's focus on interface. 

In [2]:
total_pennies = 3274
dollars = total_pennies//100
cents = total_pennies % 100

total_cash = Money(dollars,cents)


Suppose your code splits pennies into dollars and cents over and over, and you're tired of repeating this calculation.<br>
You may be able to change the constructor(\_\_init\_\_), but that means refactoring Money class.<br><br>

Some languages let you define several constructors, but Python makes you pick just one.<br>
In this case, you can usefully create factory function taking argument you want, creating and returning the object.


In [3]:
def money_from_pennies(total_cents):
    dollars = total_cents//100
    cents = total_cents % 100
    return Money(dollars,cents)

But this time, imagine that you also routinely need to parse a string like "$140.75".

In [19]:
import re
def money_from_string(amount):
    match= re.search(
        r"^\$(?P<dollars>\d+)\.?(?P<cents>\d\d)?", amount #P stands for paramter. This can be used later 
    )
    if match is None:
        raise ValueError("Invalid amount: %r"+repr(amount))
    dollars = int(match.group("dollars"))               # extract the value of the given paramter.
    cents = int(match.group("cents"))
    return Money(dollars,cents)

These are **effectively** alternative constructors: But this has some problems;
- it's awkward to have them as separate functions. 
- what happens if you subclass Money? As it returns "Money" object, it becomes worthless. 

Python solves these problems in unique way which may be absent in other languages: the **classmethod** decorator.

In [85]:
class Money:
    def __init__(self,dollars,cents):
        self.dollars = dollars
        self.cents = cents 
    
    @classmethod
    def from_pennies(cls,total_cents):
        dollars = total_cents//100
        cents = total_cents % 100
        return cls(dollars,cents) #Initialization point 1
    
    @classmethod
    def from_string(cls,amount):
        match= re.search(
        r'^\$(?P<dollars>\d+)\.(?P<cents>[0-9]{1,2})$', amount) #parameter
        
        if match is None:
            raise ValueError("Invalid amount: "+repr(amount))
        dollars = int(match.group("dollars"))
        cents = int(match.group("cents"))
        return cls(dollars,cents) #Initialization point 2

When applied to a method definition, classmethod modifies how that method is invoked and interpreted.<br>
The first argument is now the class itself. "cls" is a variable holding the **current class object**.<br>
So the last line is creating a new instance of Money in this case.<br>

As you can imagine, the real benefit of using classmethod is when you subclass them.

In [28]:
class TipMoney(Money):
    pass
tip = TipMoney.from_pennies(475)
type(tip)

__main__.TipMoney

In [29]:
tip.cents

75

In [87]:
tip2 = TipMoney.from_string("$14")
print(tip2.dollars)
print(tip2.cents)

ValueError: Invalid amount: '$14'

Plus, if you want to change the name of the Money class, you only have to change it in one place(in Money) not in the all subclasses. 

### Simple factory? or Alternative constructor?

Although this form of factory pattern is called "simple factory", it may be more appropriate to call it "alternative constructor."